In [11]:
# Libraries
import pandas as pd
from datetime import datetime, timedelta
import dateutil.parser as parser

In [50]:
# CSV Sources

properties_cols = ['property_id','latitude','longitude','localidad', 'city', 'country']
convert_pro = lambda x: (x.replace('\xad','').title())  # function used to clean the locality column, Chí­A -> Chí­a
df_properties = pd.read_csv('sources\properties.csv', usecols = properties_cols, converters = {'localidad':convert_pro})

users_cols = ['property_id','user_id']
df_users = pd.read_csv('sources\\users.csv', usecols = users_cols)

visits_cols = ['scheduled_id','property_id','begin_date','end_date','status']
convert_vis = lambda x: (parser.parse(x) + timedelta(hours=2))  # function used to change the time UTC-3 to UTC-5 (Colombian time) in datetime columns
df_visits = pd.read_csv('sources\\visits.csv', usecols = visits_cols, converters = {'begin_date':convert_vis,'end_date':convert_vis })
df_visits['visit_date'] = df_visits['begin_date'].apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))



In [10]:
API_KEY = '2DAAC28RT2V8Q2SWYQVXV3DZF'

import requests

l = ['Content-Type','Content-Length']
response = requests.get("https://api.thedogapi.com/v1/breeds/1")
response.headers.get('Content-Length')


'357'

In [8]:
response.headers

{'x-dns-prefetch-control': 'off', 'x-frame-options': 'SAMEORIGIN', 'strict-transport-security': 'max-age=15552000; includeSubDomains', 'x-download-options': 'noopen', 'x-content-type-options': 'nosniff', 'x-xss-protection': '1; mode=block', 'vary': 'Origin', 'content-type': 'application/json; charset=utf-8', 'x-response-time': '2ms', 'X-Cloud-Trace-Context': '43152d9569f810ab43c090d82934246f', 'Date': 'Sun, 17 Apr 2022 05:38:48 GMT', 'Server': 'Google Frontend', 'Content-Length': '357'}